In [ ]:
# Saved for later use
'''
all_data_sources.isnull().sum()

test_df = pd.DataFrame()
test_df["audio_features_track_id"] = lyrics_features["track_id"]
test_df["tracks_track_id"] = tracks["id"]

test_df['Match'] = test_df.apply(lambda x: 1 if x['audio_features_track_id'] in x['tracks_track_id'] else 0, axis=1)
'''

In [6]:
#libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
albums = pd.read_csv('SpotGenTrack/Data Sources/spotify_albums.csv', index_col = 0)
artists = pd.read_csv('SpotGenTrack/Data Sources/spotify_artists.csv', index_col = 0)
tracks = pd.read_csv('SpotGenTrack/Data Sources/spotify_tracks.csv', index_col = 0)
audio_features = pd.read_csv('SpotGenTrack/Features Extracted/low_level_audio_features.csv', index_col = 0)
lyrics_features = pd.read_csv('SpotGenTrack/Features Extracted/lyrics_features.csv', index_col = 0)

In [9]:
all_data_sources = tracks.merge(albums, how='left', left_on='album_id', right_on='id', suffixes=('_track','_albums')).merge(artists, how='left', left_on='artist_id', right_on='id', suffixes=('_track','_artists'))
#all_data_sources = all_data_sources.merge(audio_features, how="left", left_on="id", right_on='track_id', suffixes=('_track','_audio-features')).merge(lyrics_features, how="left", left_on="id", right_on='track_id', suffixes=('_tracks','_lyrics-features'))
df = all_data_sources

In [20]:
df.columns
df.drop(columns=["album_id", "analysis_url", "artists_id", "href_track", "id_track", "preview_url", "track_href", "track_name_prev_track", "track_number", "uri_track", "type_track",\
                 "artist_id", "external_urls", "href_albums", "id_albums", "images", "name_albums", "release_date", "release_date_precision", "total_tracks", "track_id_track", "track_name_prev_albums",\, 
                    "uri_albums", "type_albums", "id", "track_id_artists", "track_name_prev", "type"])

SyntaxError: unexpected character after line continuation character (3215079202.py, line 3)

In [21]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint

X = df.drop(columns="popularity")
y = df["popularity"]

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42, error_score="raise")
rnd_search.fit(X, y)

ValueError: could not convert string to float: '1ttVc5nDYMHWcLq2u0bKNj'